In [120]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from sklearn import cluster
from kneed import KneeLocator

In [121]:
#read the pkl file for movies
df = pd.read_pickle('./data/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./ABC/full_shows.pkl')

In [122]:
df

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
11,11,Family,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,[],"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,NaN,,Ballet Now
35,35,Family,Zog,NaN,Zog,"Based on the picture book by Julia Donaldson, ...","Based on the picture book by Julia Donaldson, ...","[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4feature, 4featured, abcme-shoulder...",2020-04-17 11:45:30,G,1621.0,NaN,,Zog
111,111,Family,Armadillo: Narrated By David Attenborough,NaN,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,In the heart of Brazil lives an animal so elus...,"[ABC TV, DOCUMENTARY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[Host Sir David Attenborough],"[abc1, uk, docs, factual, nature, wildlife, en...",2021-09-19 19:41:30,PG,3134.0,NaN,,Armadillo: Narrated By David Attenborough
137,137,Family,Stick Man,NaN,Stick Man,Stick Man lives in the family tree with his St...,Stick Man lives in the family tree with his St...,"[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","[abc4kids, family-viewing, 4xmas-features, abc...",2020-04-17 11:44:30,G,1626.0,NaN,,Stick Man
158,158,Family,The Snail And The Whale,NaN,The Snail And The Whale,"A beautiful tale of an unlikely friendship, a ...","A beautiful tale of an unlikely friendship, a ...","[ABC Kids, ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW177...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4featured, 4feature, 4family, 4uppe...",2020-04-17 11:46:30,G,1601.0,NaN,,The Snail And The Whale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,2255,Education,What's For Dinner? Lunar New Year (Simplified ...,NaN,What's For Dinner? Lunar New Year (Simplified ...,Welcome to Penny's place for Lunar New Year! I...,Welcome to Penny's place for Lunar New Year! I...,"[ABC ME, ABC TV, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/iv/IV210...,[],"[abc3, education, iview, abc1, family-viewing]",2021-02-05 06:00:00,G,783.0,NaN,,What's For Dinner? Lunar New Year (Simplified ...
2305,2305,Education,BTN: Antarctica Special,NaN,BTN: Antarctica Special,A BTN Special all about Antarctica; from the p...,A BTN Special all about Antarctica from the pe...,"[ABC ME, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/ne/NE190...,[],"[abc3, education, primary-humanities, secondar...",2019-09-24 10:00:00,NaN,1228.0,NaN,,BTN: Antarctica Special
2354,2354,Education,Life In Medieval Europe,NaN,Life In Medieval Europe,"Set in 1350, this medieval drama unfolds throu...","Set in 1350, this medieval drama unfolds throu...","[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW027...,[],"[abc3, education, secondary-history]",2020-03-16 09:00:00,G,1475.0,NaN,,Life In Medieval Europe
2359,2359,Education,Montserrat: Living With Volcanoes,NaN,Montserrat: Living With Volcanoes,This program deals with the impacts of the vol...,This program deals with the impacts of the vol...,"[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW113...,[],"[abc3, education, secondary-geography]",2017-08-17 11:00:00,G,2750.0,NaN,,Montserrat: Living With Volcanoes


In [123]:
df_2

,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
title,,,,,,,,,,,
199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re..."
7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ..."
7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ..."
7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]"
72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]"
...,...,...,...,...,...,...,...,...,...,...,...
hoopla doopla!,12201,Kids,hoopla doopla! is a live action preschool show...,[Jango dresses up as a king and his friends mi...,[],2022-01-28 06:15:10,G,"[720.0, 720.0, 720.0, 720.0, 720.0, 720.0, 720...","[4015, 4097, 4184, 4189, 4436, 4443, 4821, 483...",[ABC Kids],"[ps-birthday, 4sing-dance, 4play-time, 4xmas, ..."
hoopla doopla! (Mandarin),12202,Kids,hoopla doopla! is a live action preschool show...,[Jango is determined to be picked to be in the...,[],2022-01-28 06:35:20,G,"[720.0, 720.0, 720.0, 720.0, 720.0, 720.0, 720...","[3902, 4111, 4124, 4140, 4252, 4263, 4432, 445...",[ABC Kids],"[abc4kids, education, 4liveaction, forever]"
storyTree,12203,Kids,"Join storyteller, Miss Treeleaf, in an imagina...","[When little red hen bakes a cake, her lazy fr...",[],2020-03-20 14:11:00,G,"[210.0, 210.0, 210.0, 210.0, 210.0, 210.0, 210...","[3968, 4046, 4053, 4266, 4274, 4363, 4568, 472...","[ABC Kids, EDUCATION]","[4halloween, 4story, abc4kids, education, prim..."


In [124]:
#read json file activity for movies
activity = pd.read_json('./activities.json')

In [125]:
activity.head()
activity.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,1577,1577,1577
2,1540,1540,1540
3,1640,1640,1640
4,1627,1627,1627
5,1616,1616,1616


## TF-IDF + kmeans clustering

In [126]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


13


In [127]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

9


In [128]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [129]:
# Cluster the documents using KMeans
num_clusters_shows = 9
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
0,199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re...",8
1,7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ...",7
2,7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ...",1
3,7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]",6
4,72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",2


In [130]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'

In [131]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
237,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet,6
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,5
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,10
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,2
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,5


In [132]:
expat_library_shows = df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.shape

(23, 13)

In [133]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
0,199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re...",8
1,7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ...",7
2,7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ...",1
3,7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]",6
4,72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",2


In [134]:
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
56,Australia Debates,11585,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ...",8
57,Australia Remastered: Australia's Oceans,11586,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[Host Aaron Pedersen],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[1933, 2047, 2104]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[a:retirees, docs, wildlife, ns:be-informed, i...",6
58,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster...",8
59,Australia Remastered: Nature's Great Divide,11588,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[Host Aaron Pedersen],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[2840, 3476, 3601]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, docs, environment, family-viewing, wil...",8
60,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:...",6


In [135]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: family new home story life school stick christmas lives play chinese love ultimately
Cluster 1: sarah narrated portrait music artistic new intimate artist walks terence stamp priscilla dignity
Cluster 2: special episode school play deep research pearl gadabout zog king evidence iii remains
Cluster 3: magical takes london ancient geography program animals dancers history night reveals world mouse
Cluster 4: beach stranded summer lost time kangaroo cadets deserted gave neville flashback recalls day
Cluster 5: disability story australian australia man explores iconic ballet people extraordinary artists international life
Cluster 6: love isolated performed story follow creates metal world island australian ballet orchestra art
Cluster 7: seafood freshest welcome dumplings homemade markets salad colourful tossing tradition dragons starts fish
Cluster 8: year old age gospel life jane meets inspired abc love singer best begin
Cluster 9: book julia donaldson based world tour scheffl

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [136]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: family songs join learning including best new adventures fun help range life series
Cluster 1: time way life election work adventures live business changed new friends family follow
Cluster 2: series winning award stories words life years real people just told new telling
Cluster 3: old year world kids animals big takes friends school new adventures adventure girl
Cluster 4: stage science english series early australian minute lessons teachers teach stages skills geography
Cluster 5: set comedy action series inspired town live physical features mandarin life play preschool
Cluster 6: australian journey love explore place books embarks world dr identity imagination brilliant celebrate
Cluster 7: news abc stories issues brings reporting today original investigations exclusive presented matter live
Cluster 8: australia world people home country share make series australians new life stories national


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [137]:
joined_activity = pd.merge(df, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
1,311.0,Drama,Trust The Man,"Two New York couples; a successful actress, he...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,[Director Bart Freundlich],"'abc1', 'abc2', 'comedy', 'drama', 'film', 'ro...",0.0,311,Like,3
3,141.0,Drama,The Last Station,A historical drama that illustrates Russian au...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW278...,[Director Michael Hoffman],"'abc1', 'abc2', 'film', 'drama', 'romance', 'a...",11.0,141,Play,3
17,590.0,Drama,Re-Frame 2020,More than 4 million Australians live with a di...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Bridie McKim],"'abc1', 'disability', 'arts', 'abcarts', 'dram...",5.0,590,Like,4
28,383.0,Drama,Mr Smith Goes To Washington,When a naive young man is appointed to the Uni...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,[Director Frank Capra],"'abc1', 'abc2', 'drama', 'comedy-drama', 'poli...",5.0,383,Play,1
40,895.0,Drama,Suburban Mayhem,When her father threatens to contact social se...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[Director Paul Goldman],"'abc1', 'abc2', 'film', 'drama', 'comedy', 'cr...",5.0,895,Like,2


In [223]:
joined_activity_shows = pd.merge(df_2, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,title,id,category,description,directors_actors,episodes_ids,tags2,cluster,content_id,activity,user_id
3007,Finding Joy,11720.0,Comedy,"Joy is happy with her life, things are fine th...","[Lochlann O'Mearain, Cast Amy Huberman, Hannah...","[8261, 8278, 8358, 8538, 8780, 8781, 8808, 881...","'romance', 'ireland', 'abc2', 'dating', 'charm...",1.0,11720,Play,3
3012,First Nation Farmers,11725.0,News,For millennia Aboriginal and Torres Strait Isl...,[],"[733, 958, 1205, 1575, 1660]","'farming', 'news24', 'secondary-maths', 'iview...",8.0,11725,Play,5
3015,Kiri and Lou,11821.0,Family,"Kiri and Lou follows the adventures of Kiri, a...",[],"[1800, 1801, 1877, 1888, 1950, 2053, 2055, 219...","'dinosaurs', 'primary', 'play', 'creatures', '...",0.0,11821,Play,5
3019,David Attenborough's Micro Monsters,11681.0,Family,Micro Monsters is a thrilling voyage into a re...,[Host Sir David Attenborough],"[2905, 2947, 3091]","'abc2', 'abcme-shoulder', 'docs', 'science', '...",8.0,11681,Play,5
3026,Sarah's Channel,12006.0,Comedy,Online Beauty Vlogger Sarah knows how to creat...,"[Anna Houston, Genevieve Lemon, Nick Coyle, Ca...","[8196, 8607, 8609, 8695, 9117, 9370]","'abc2', 'quirky', 'featured-comedy', 'short-co...",0.0,12006,Dislike,5


In [176]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
2114,895.0,Drama,Suburban Mayhem,When her father threatens to contact social se...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[Director Paul Goldman],"'abc1', 'abc2', 'film', 'drama', 'comedy', 'cr...",5.0,895,Like,1
995,907.0,Drama,Greetings From Tim Buckley,"In 1991, as young Jeff Buckley prepares to per...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW289...,[Director Daniel Algrant],"'abc1', 'abc2', 'usa', 'film', 'feature-length...",5.0,907,Like,1
890,523.0,Drama,Sisters Of War,Inspired by the remarkable true story of two e...,https://cdn.iview.abc.net.au/thumbs/i/dr/DR090...,[Director Brendan Maher],"'abc1', 'drama', 'history', 'a:established-par...",5.0,523,Like,1
1554,105.0,Kids,Thomas And Friends: Big World! Big Adventures!...,Join Thomas as he embarks on an epic journey a...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW137...,[],"'abc4kids', '4machines', '4action', '4thomas',...",0.0,105,Like,1
1694,209.0,Education,"Tell Your Story, Change Your World","Only you can tell your own story, but getting ...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK189...,[],"'abc3', 'education', 'secondary-english'",12.0,209,Like,1
...,...,...,...,...,...,...,...,...,...,...,...
1968,993.0,Documentary,Hitsville: The Making Of Motown,Hitsville: The Making of Motown is the story o...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[],"'abc2', 'docs', 'usa', 'arts', 'culture', 'mus...",10.0,993,Like,5
744,990.0,Drama,Creation,From the writer of Master and Commander comes ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW288...,[Director Jon Amiel],"'abc1', 'abc2', 'drama', 'romance', 'film', 'b...",5.0,990,Like,5
1839,741.0,Documentary,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[Host Julia Zemiro, Brian Cox]","'abc1', 'science', 'docs', 'space', 'family-vi...",9.0,741,Like,5
2536,190.0,Arts,Songs of Praise: Enduring Hymns,"Wynne Evans, Melanie Marshall, Wallingford Par...",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX899...,[],"'abc1', 'uk', 'religion', 'arts', 'music', 'cu...",0.0,190,Like,5


In [224]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,title,id,category,description,directors_actors,episodes_ids,tags2,cluster,content_id,activity,user_id
3036,"If You See It, You Can Be It",11787.0,Education,What do you want to be when you grow up? No ma...,[],"[12816, 13261, 13689, 13915, 14109, 14176, 142...","'secondary-health', 'abc3', '3science-week', '...",4.0,11787,Like,1
4698,Growing Up In The Early 1900s,11759.0,Education,What was life like for working-class children ...,[],"[12708, 13464]","'primary-humanities', 'abc3', 'education'",6.0,11759,Like,1
4751,Australia Remastered: Wild Wars,11591.0,Family,Life in Australia is a battle. Shaped by the c...,[Host Aaron Pedersen],"[2364, 3015, 3501, 3640]","'a:retirees', 'docs', 'wildlife', 'science-wee...",3.0,11591,Like,1
7207,Mini Lessons: English,11877.0,Education,"In these ten-minute lessons, Australian teache...",[],"[12659, 12665, 12864, 12874, 13034, 13128, 131...","'abc3', 'education', 'primary-english'",4.0,11877,Like,1
7195,SeaChange,12012.0,Drama,"In one day successful city lawyer, Laura Gibso...","[Kevin Harrington, Sigrid Thornton, David Wenh...","[10579, 10593, 10628, 10668, 10681, 10687, 107...","'abc2', 'soap', 'love', 'coast', 'compelling',...",3.0,12012,Like,1
...,...,...,...,...,...,...,...,...,...,...,...
4619,Planet America,11949.0,News,,[],"[644, 686, 703, 1327, 1433, 1436]","'planet', 'ameria', 'critical', 'theory', 'rac...",0.0,11949,Like,5
7264,So You Want To Be A Designer?,12031.0,Education,See how designers create products that meet us...,[],"[12972, 13363, 13859, 14367]","'secondary-technology', 'abc3', 'education'",0.0,12031,Like,5
4623,Dragons: Riders Of Berk,11699.0,Family,Follows the adventures of master dragon traine...,[],"[2572, 3407, 3804]","'3fantasy', 'abc3', '3action', '3toons', '3wil...",6.0,11699,Like,5
5715,Teletubbies,12051.0,Kids,Teletubbies stars iconic characters; Tinky Win...,[],"[3849, 4196, 4197, 4284, 4403, 4405, 4566, 468...","'4silly', '4gentle', '4liveaction', 'abc4kids'...",6.0,12051,Like,5


In [203]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,1,5.0
1,1,0.0
2,1,3.0
3,2,0.0
4,2,6.0
5,2,1.0
6,3,0.0
7,3,5.0
8,3,10.0
9,4,0.0


In [225]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,1,4.0
1,1,6.0
2,1,8.0
3,2,8.0
4,2,2.0
5,2,3.0
6,3,1.0
7,3,8.0
8,3,3.0
9,4,1.0


In [218]:
#filter expat_data_likes dataframe by another dataframe with two columns: user_id and cluster
#this is only activity 'Like' for top-3 clusters for each user
# expat_data_top = pd.merge(expat_data_likes, top3, on=['user_id', 'cluster'])
# expat_data_top.sort_values(by=['user_id'])

In [227]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,5.0,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?
1,1,5.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
2,1,5.0,1385,Family,Coppelia,NaN,Coppelia,"A sparkling tale of magic and mischief, Coppel...","A sparkling tale of magic and mischief, Coppel...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,6511.0,NaN,,Coppelia
3,1,5.0,32,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon
4,1,5.0,44,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[Host Moataz Hamde],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,5,0.0,709,Arts,Deadly Family Portraits: Electric Mimili,NaN,Deadly Family Portraits: Electric Mimili,Robert Fielding and Zaachariaha Fielding tell ...,Robert Fielding and Zaachariaha Fielding tell ...,"[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC183...,[],"[abc1, abcarts, indigenious, docs, arts, cultu...",2019-08-09 07:00:00,G,555.0,NaN,,Deadly Family Portraits: Electric Mimili
192,5,0.0,1958,Education,Deadly Family Portraits: Electric Mimili,NaN,Deadly Family Portraits: Electric Mimili,Robert Fielding and Zaachariaha Fielding tell ...,Robert Fielding and Zaachariaha Fielding tell ...,"[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC183...,[],"[abc1, abcarts, indigenious, docs, arts, cultu...",2019-08-09 07:00:00,G,555.0,NaN,,Deadly Family Portraits: Electric Mimili
193,5,8.0,367,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[Host Steve Waugh],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India
194,5,8.0,375,Documentary,New Dads,NaN,New Dads,Jane Caro meets five Australian men navigating...,Jane Caro meets five Australian men navigating...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[Host Jane Caro],"[abc1, australia, factual, docs, people, real-...",2019-10-17 15:00:00,PG,3346.0,NaN,,New Dads


In [228]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,4.0,Australia's Prime Ministers,11593,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
1,1,6.0,Australia Remastered: Australia's Oceans,11586,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[Host Aaron Pedersen],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[1933, 2047, 2104]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[a:retirees, docs, wildlife, ns:be-informed, i..."
2,1,6.0,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
3,1,6.0,Australia's Ocean Odyssey: A Journey Down The ...,11592,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[Narrator Marta Dusseldorp],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[1967, 2009, 3463, 3832]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, abcme-shoulder, docs, environment, abc..."
4,1,6.0,Australian Of The Year Profiles 2022,11594,Arts,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[],2022-01-05 17:00:00,G,"[197.0, 209.0, 210.0, 198.0, 181.0, 196.0, 210...","[11795, 11798, 11800, 11813, 11828, 11861, 118...","[ABC TV, DOCUMENTARY]","[western-australia, victoria, docs, disability..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,5,6.0,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
73,5,6.0,Australia's Ocean Odyssey: A Journey Down The ...,11592,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[Narrator Marta Dusseldorp],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[1967, 2009, 3463, 3832]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, abcme-shoulder, docs, environment, abc..."
74,5,6.0,Australian Of The Year Profiles 2022,11594,Arts,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[],2022-01-05 17:00:00,G,"[197.0, 209.0, 210.0, 198.0, 181.0, 196.0, 210...","[11795, 11798, 11800, 11813, 11828, 11861, 118...","[ABC TV, DOCUMENTARY]","[western-australia, victoria, docs, disability..."
75,5,6.0,Miriam Margolyes Almost Australian,11883,Documentary,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,[Host Miriam Margolyes],2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]","[10099, 10332, 10510]","[ABC TV, DOCUMENTARY]","[a:retirees, revealing, binge-worthy, docs, pe..."


In [221]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,5.0,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?
1,1,5.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
2,1,5.0,1385,Family,Coppelia,NaN,Coppelia,"A sparkling tale of magic and mischief, Coppel...","A sparkling tale of magic and mischief, Coppel...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,6511.0,NaN,,Coppelia
3,1,5.0,32,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon
4,1,5.0,44,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[Host Moataz Hamde],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,5,0.0,709,Arts,Deadly Family Portraits: Electric Mimili,NaN,Deadly Family Portraits: Electric Mimili,Robert Fielding and Zaachariaha Fielding tell ...,Robert Fielding and Zaachariaha Fielding tell ...,"[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC183...,[],"[abc1, abcarts, indigenious, docs, arts, cultu...",2019-08-09 07:00:00,G,555.0,NaN,,Deadly Family Portraits: Electric Mimili
192,5,0.0,1958,Education,Deadly Family Portraits: Electric Mimili,NaN,Deadly Family Portraits: Electric Mimili,Robert Fielding and Zaachariaha Fielding tell ...,Robert Fielding and Zaachariaha Fielding tell ...,"[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC183...,[],"[abc1, abcarts, indigenious, docs, arts, cultu...",2019-08-09 07:00:00,G,555.0,NaN,,Deadly Family Portraits: Electric Mimili
193,5,8.0,367,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[Host Steve Waugh],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India
194,5,8.0,375,Documentary,New Dads,NaN,New Dads,Jane Caro meets five Australian men navigating...,Jane Caro meets five Australian men navigating...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[Host Jane Caro],"[abc1, australia, factual, docs, people, real-...",2019-10-17 15:00:00,PG,3346.0,NaN,,New Dads


In [229]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,4.0,Australia's Prime Ministers,11593,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
2,1,6.0,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
6,1,6.0,Prepping Australia,11966,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[9481, 9684, 9738]","[ABC TV Plus, DOCUMENTARY]","[abc2, conspiracy, offbeat, docs, environment,..."
7,1,8.0,Australia Debates,11585,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
8,1,8.0,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
12,1,8.0,Gardening Australia - My Garden Path,11741,Arts,Gardening Australia is hosted by Costa Georgia...,[We meet a garden designer who celebrates text...,[],2021-02-12 07:00:00,G,"[366.0, 372.0, 407.0, 509.0, 385.0, 467.0, 401...","[11753, 11758, 11776, 11787, 11794, 11820, 118...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[western-australia, victoria, darwin, regional..."
15,1,8.0,The Art Of Australia,12055,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[Host Edmund Capon],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[9720, 10142, 10214]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[a:retirees, revealing, indigenous-history, do..."
16,1,8.0,The Great Australian Bee Challenge,12078,Family,"In this two-part special, host Paul West chall...","[In the first of two programs, host Paul West ...",[Host Paul West],2019-01-29 21:33:06,G,"[3594.0, 3336.0]","[2169, 2767]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[a:retirees, revealing, docs, wildlife, compel..."
19,2,8.0,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
20,2,8.0,Australia Remastered: Nature's Great Divide,11588,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[Host Aaron Pedersen],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[2840, 3476, 3601]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, docs, environment, family-viewing, wil..."


In [222]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,5.0,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?
1,1,5.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
35,1,0.0,331,Kids,Play School: Walking Together,NaN,Play School: Walking Together,"Hunter, Rachael & Emma Donovan walk us through...","Hunter, Rachael & Emma Donovan walk us through...",[ABC Kids],https://cdn.iview.abc.net.au/thumbs/i/ck/CK201...,[],[abc4kids],2021-05-27 09:27:03,G,1579.0,NaN,,Play School: Walking Together
36,1,0.0,390,Documentary,Christmas Sounds Better This Year,NaN,Christmas Sounds Better This Year,Every Christmas the same old songs are played ...,Every Christmas the same old songs are played ...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[Host Kumi Taguchi],"[abc1, australia, docs, factual, music, people...",2019-12-17 22:35:13,G,3538.0,NaN,,Christmas Sounds Better This Year
43,2,0.0,331,Kids,Play School: Walking Together,NaN,Play School: Walking Together,"Hunter, Rachael & Emma Donovan walk us through...","Hunter, Rachael & Emma Donovan walk us through...",[ABC Kids],https://cdn.iview.abc.net.au/thumbs/i/ck/CK201...,[],[abc4kids],2021-05-27 09:27:03,G,1579.0,NaN,,Play School: Walking Together
44,2,0.0,390,Documentary,Christmas Sounds Better This Year,NaN,Christmas Sounds Better This Year,Every Christmas the same old songs are played ...,Every Christmas the same old songs are played ...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[Host Kumi Taguchi],"[abc1, australia, docs, factual, music, people...",2019-12-17 22:35:13,G,3538.0,NaN,,Christmas Sounds Better This Year
50,2,6.0,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet
52,2,6.0,1042,Drama,An Accidental Soldier,NaN,An Accidental Soldier,"On the Western Front in WWI, an Australian on ...","On the Western Front in WWI, an Australian on ...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR120...,[Director Rachel Ward],"[abc1, drama, film, war, love, romance, adapta...",2021-06-11 07:00:00,M,5502.0,NaN,,An Accidental Soldier
55,2,1.0,122,Documentary,Cyber Dreaming,NaN,Cyber Dreaming,Graphic artist Stuart Campbell has developed h...,Graphic artist Stuart Campbell has developed h...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/ac/publi...,[],"[abc1, abcarts, arts, docs, aussie, naidoc, hi...",2018-07-11 14:00:50,G,1570.0,NaN,,Cyber Dreaming
56,2,1.0,760,Documentary,Bird Nerd: The Art Of Leila Jeffreys,NaN,Bird Nerd: The Art Of Leila Jeffreys,This intimate portrait follows beloved Austral...,This intimate portrait follows beloved Austral...,"[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/ac/AC180...,[],"[abc1, australia, docs, factual, abcarts, arts...",2019-10-06 15:00:38,G,1518.0,NaN,,Bird Nerd: The Art Of Leila Jeffreys


In [230]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,4.0,Australia's Prime Ministers,11593,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
2,1,6.0,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
6,1,6.0,Prepping Australia,11966,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[9481, 9684, 9738]","[ABC TV Plus, DOCUMENTARY]","[abc2, conspiracy, offbeat, docs, environment,..."
7,1,8.0,Australia Debates,11585,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
8,1,8.0,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
19,2,8.0,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
20,2,8.0,Australia Remastered: Nature's Great Divide,11588,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[Host Aaron Pedersen],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[2840, 3476, 3601]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, docs, environment, family-viewing, wil..."
29,2,2.0,Australian Story,11595,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[623, 626, 633, 638, 647, 653, 671, 673, 695, ...","[ABC TV, ABC NEWS, DOCUMENTARY]","[inside-invictus, current-affairs, informative..."
30,2,3.0,Australia Remastered: Wild Wars,11591,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[Host Aaron Pedersen],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[2364, 3015, 3501, 3640]","[ABC TV, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
33,3,8.0,Australia Debates,11585,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
